# MODELS

In [ ]:
pip install ultralytics opencv-python

#### detecttion (coco, open image v7)

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt") # nano
model = YOLO("yolov8s.pt") # small
model = YOLO("yolov8m.pt") # medium
model = YOLO("yolov8l.pt") # large
model = YOLO("yolov8x.pt") # extra large

data = "coco.yaml"

# document
# https://docs.ultralytics.com/tasks/detect/

#### segmentation (coco)

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n-seg.pt") # nano
model = YOLO("yolov8s-seg.pt") # small
model = YOLO("yolov8m-seg.pt") # medium
model = YOLO("yolov8l-seg.pt") # large
model = YOLO("yolov8x-seg.pt") # extra large

data = "coco.yaml"
# or data = "coco-seg.yaml" --> lấy kết quả train từ coco.yaml :)) chứ cái data này bé tí

# document
# https://docs.ultralytics.com/tasks/segment/

#### pose (coco)

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n-pose.pt") # nano
model = YOLO("yolov8s-pose.pt") # small
model = YOLO("yolov8m-pose.pt") # medium
model = YOLO("yolov8l-pose.pt") # large
model = YOLO("yolov8x-pose.pt") # extra large

data = "coco-pose.yaml" # 1 class: person

# document
# https://docs.ultralytics.com/tasks/pose/

#### obb (DOTAv1)

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n-obb.pt") # nano
model = YOLO("yolov8s-obb.pt") # small
model = YOLO("yolov8m-obb.pt") # medium
model = YOLO("yolov8l-obb.pt") # large
model = YOLO("yolov8x-obb.pt") # extra large

# https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/datasets/DOTAv1.yaml
data = "DOTAv1.yaml" # 15 class

# document
# https://docs.ultralytics.com/tasks/obb/

#### classification (ImageNet)

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n-cls.pt") # nano
model = YOLO("yolov8s-cls.pt") # small
model = YOLO("yolov8m-cls.pt") # medium
model = YOLO("yolov8l-cls.pt") # large
model = YOLO("yolov8x-cls.pt") # extra large

# https://docs.ultralytics.com/datasets/classify/imagenet/
data = "imagenet"

# document
# https://docs.ultralytics.com/tasks/classify/

# train model

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")

# Use the model
model.train(data="coco8.yaml", epochs=3)  # train model
metrics = model.val()  # đánh giá hiệu suất mô hình trên tập xác thực

# ONLY TRACKING - LINES

In [ ]:
from collections import defaultdict

import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")

# Open the video file
video_path = "/kaggle/input/video-img-tracking/shop2.mp4"
cap = cv2.VideoCapture(video_path)

# Store the track history
track_history = defaultdict(lambda: [])

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define output video path
output_video_path = "output_video1.mp4"

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 255, 255), thickness=5)

        # Write the annotated frame to the output video
        output_video.write(annotated_frame)

#         # Break the loop if 'q' is pressed
#         if cv2.waitKey(1) & 0xFF == ord("q"):
#             break
#     else:
#         # Break the loop if the end of the video is reached
#         break

# Release the video capture object, release the VideoWriter object, and close the display window
cap.release()
output_video.release()

# TRACKING - COUNTING - LINES

In [ ]:
from collections import defaultdict

import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")

# Open the video file
video_path = "/kaggle/input/video-img-tracking/Vehicles.mp4"
cap = cv2.VideoCapture(video_path)

# Store the track history
track_history = defaultdict(lambda: [])

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define output video path
output_video_path = "output_video11.mp4"

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Initialize variables for counting vehicles
vehicle_count = 0
prev_frame_vehicles = set()

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Store current frame's vehicles
        curr_frame_vehicles = set(track_ids)

        # Count new vehicles
        new_vehicles = curr_frame_vehicles - prev_frame_vehicles
        vehicle_count += len(new_vehicles)

        # Update previous frame's vehicles
        prev_frame_vehicles = curr_frame_vehicles

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 255, 255), thickness=5)


        # Draw a transparent black box for the text
        box_coordinates = ((50, 20), (350, 80))  # Top-left and bottom-right coordinates
        cv2.rectangle(annotated_frame, box_coordinates[0], box_coordinates[1], (0, 0, 0, 128), -1)  # Fill with black

        # Write vehicle count on the frame
        cv2.putText(annotated_frame, f"Vehicle Count: {vehicle_count}", (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 3)

        # Write the annotated frame to the output video
        output_video.write(annotated_frame)

#         # Break the loop if 'q' is pressed
#         if cv2.waitKey(1) & 0xFF == ord("q"):
#             break
#     else:
#         # Break the loop if the end of the video is reached
#         break

# Print the total number of vehicles
print("Total vehicles detected:", vehicle_count)

# Release the video capture object, release the VideoWriter object, and close the display window
cap.release()
output_video.release()


# HEATMAP (ORIGINAL)

In [ ]:
import cv2
from ultralytics import YOLO, solutions

# Load YOLO model
model = YOLO("yolov8n.pt")

# Open video file
cap = cv2.VideoCapture("/kaggle/input/video-img-tracking/tenis6.mp4")
assert cap.isOpened(), "Error reading video file"

# Get video properties
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Initialize video writer
video_writer = cv2.VideoWriter("heatmap_output1.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Initialize heatmap object
heatmap_obj = solutions.Heatmap(
    colormap=cv2.COLORMAP_PARULA,
    view_img=True,
    shape="circle",
    classes_names=model.names,
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Perform tracking on the current frame
    tracks = model.track(im0, persist=True, show=False)

    # Generate heatmap on the frame
    im0 = heatmap_obj.generate_heatmap(im0, tracks)

    # Write the frame to the output video
    video_writer.write(im0)

# Release resources
cap.release()
video_writer.release()

# HEATMAP (FROM DAT)

In [ ]:
# Import Dependences
from ultralytics import YOLO, solutions
from ultralytics.solutions import heatmap
import imutils
import cv2 as cv
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os  # Import thư viện os

def track_createHeatmap(video_path, output_path, heatmap_output_path):
    writer = None
    (W, H) = (None, None)
    weight_path = "yolov8x.pt"

    # Kiểm tra sự tồn tại của tệp trọng số
    if not os.path.exists(weight_path):
        print(f"Weight file not found: {weight_path}")
        return

    model = YOLO(weight_path)

    # read video
    vs = cv.VideoCapture(video_path)
    if not vs.isOpened():
        print(f"Failed to open video: {video_path}")
        return

    classes_names = ["person"]
    (grabbed, frame) = vs.read()
    if not grabbed:
        print("Failed to read video")
        return
    h, w, c = frame.shape

    # Init heatmap
    heatmap_obj = heatmap.Heatmap(
        colormap=cv.COLORMAP_PARULA,
        imw=w,
        imh=h,
        view_img=False,  # Set to False for overall heatmap generation
        view_in_counts=True,
        shape="circle",
        decay_factor=1,  # No decay to keep the tracks visible
        classes_names=model.names,
    )

    # Initialize the heatmap accumulator
    heatmap_accumulator = np.zeros((h, w), dtype=np.float32)

    while True:
        # read the next frame from the file
        (grabbed, frame) = vs.read()
        if not grabbed:
            break
        tracks = model.track(frame, persist=True)
        if not tracks:
            continue  # Skip frames with no detections
        for item in tracks:
            if not item:  # Skip empty detections
                continue
            print("got it:-", item[0].boxes)
        try:
            results = heatmap_obj.extract_results(tracks)
        except AttributeError as e:
            print(f"Error extracting results: {e}")
            continue
        print("result##:-", results)
        heatmap_frame = heatmap_obj.generate_heatmap(frame, tracks)

        # Accumulate heatmap results
        heatmap_accumulator += heatmap_obj.heatmap

        if writer is None:
            # initialize our video writer
            fourcc = cv.VideoWriter_fourcc(*"mp4v")
            writer = cv.VideoWriter(output_path, fourcc, 24,
                                    (frame.shape[1], frame.shape[0]), True)
        writer.write(heatmap_frame)

    vs.release()
    if writer is not None:
        writer.release()

    # Normalize the accumulated heatmap
    heatmap_accumulator = cv.normalize(heatmap_accumulator, None, 0, 255, cv.NORM_MINMAX)
    heatmap_accumulator = np.uint8(heatmap_accumulator)
    heatmap_color = cv.applyColorMap(heatmap_accumulator, cv.COLORMAP_JET)

    # Save the final heatmap image
    cv.imwrite(heatmap_output_path, heatmap_color)

# Gọi trực tiếp hàm track_createHeatmap với đường dẫn cụ thể của video
video_path = "/content/shop2.mp4"
output_path = "shop2_heatmap1.mp4"
heatmap_output_path = "shop2_heatmap1.png"

track_createHeatmap(video_path, output_path, heatmap_output_path)

# hmm

# đóng zip thư mục

In [ ]:
!zip -r outputs.zip /content/